# Environmental Solutions Lab: Adaptive Infrastructure
## Tasks I - III: Data preparation
Intelligent Earth Center for Doctoral Training <br>
Friday 13th December 2024 <br>

Instructors:
* <alison.peard@ouce.ox.ac.uk><br>
* <yu.mo@chch.ox.ac.uk>


In [ ]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

WD = os.path.join(os.getcwd(), "data")

| Dataset | Download URL |
| ------- | ------------ |
| IBTrACs | https://www.ncei.noaa.gov/products/international-best-track-archive |
| IBTrACs metadata (a) | https://climada-python.readthedocs.io/en/stable/_modules/climada/hazard/tc_tracks.html |
| IBTrACs metadata (b) | https://doi.org/10.1175/2009BAMS2755.1 |
| Blackout | https://blackmarble.gsfc.nasa.gov |
| Natural Earth Cultural| https://www.naturalearthdata.com/downloads/10m-cultural-vectors |


### Task I: Clean IBTrACS data for storm characteristics input
The IBTrACS (International Best Track Archive for Climate Stewardship) is a comprehensive dataset for tropical cyclones. You will clean and format this dataset for input into predictive models. The steps include:
* Formatting: Review the source file for inconsistencies or formatting errors. Ensure the data is in a consistent and analysable format.
* Standardizing Records Across Agencies: IBTrACS consolidates data from multiple agencies. Harmonize these variables to ensure uniformity, such as converting all wind speed measurements to the same standard.


In [ ]:
IBTRACS = "ibtracs.since1980.list.v04r00.csv"
AGENCIES = "wind_factors.csv"

COLUMNS_IN = ["SID", "SEASON", "NUMBER","BASIN","NAME","NATURE","ISO_TIME",
           "LAT","LON","WMO_WIND","WMO_PRES","WMO_AGENCY",
           "DIST2LAND","LANDFALL",'USA_WIND','USA_RMW','REUNION_WIND','REUNION_RMW',
           'BOM_WIND',"BOM_RMW","STORM_SPEED"]

COLUMNS_OUT = ["SID","SEASON","NUMBER","BASIN","NAME","NATURE","ISO_TIME",
               "LAT","LON","WMO_WIND","WMO_PRES","STORM_SPEED","DIST2LAND",
               "LANDFALL"]

In [ ]:
# load data
ibtracs = pd.read_csv(os.path.join(WD, IBTRACS), 
skiprows=[1]) # second row is units
ibtracs.head()

In [ ]:
# check longitude range
print("Longitude range:", ibtracs['LON'].min(), ibtracs['LON'].max()) # ibtracs['LON'].max()
print("Latitude range:", ibtracs['LAT'].min(), ibtracs['LAT'].max())

The longitude range looks strange, longitude is usually in (-180, 180] or [0, 360] while latitude is usually in [-90, 90] or [0, 180]. We solve this by simply subtracting 360 from any longitudes greater than 180.

In [ ]:
def fix_longitude(lon):
    if lon > 180:
        return lon - 360
    return lon

ibtracs['LON'] = ibtracs['LON'].apply(fix_longitude)
print("Longitude range:", ibtracs['LON'].min(), ibtracs['LON'].max()) 

In [ ]:
# there are some blank rows in WMO_WIND
ibtracs = ibtracs[ibtracs["WMO_WIND"] != ' '].copy()
ibtracs['WMO_WIND'] = ibtracs['WMO_WIND'].astype(float)
ibtracs = ibtracs[COLUMNS_IN].copy()

The World Meteorolical Organisation (WMO) defines the _maximum sustained wind_ as the 10-minute average wind speed 10m above ground level ($U_{10}$).

Despite this, many organisations average over one-minute or three-minute periods. [Knapp et al (2010)](https://journals.ametsoc.org/view/journals/mwre/138/4/2009mwr3123.1.xml) provide a table of shift and scale factors to convert different agencies to the standard 10-minute maximum sustained wind.

We will use a dataset of shift and scale factors to standardise our wind records.

In [ ]:
wind_factors = pd.read_csv(os.path.join(WD, AGENCIES), index_col=0)
wind_factors.head()

In [ ]:
def convert_winds(row:pd.Series) -> float:
    agency = row['WMO_AGENCY']
    scale = wind_factors.loc[agency, 'scale']
    shift = wind_factors.loc[agency, 'shift']
    new_wind = row['WMO_WIND'] * scale + shift
    return new_wind

ibtracs['WMO_WIND'] = ibtracs.apply(convert_winds, axis=1)

In [ ]:
ibtracs[COLUMNS_OUT].to_csv(os.path.join(WD, "ibtracs_formatted"), index=False)

### Task II: Get storm information for blackout events
The blackout dataset, derived from sources like the Black Marble project, captures key details about power outages, including duration, affected customers, and geographic locations. To integrate storm-related information into this dataset, complete the following steps:
* Matching records: match the IBTrACS data with the blackout data by a common identifier, i.e., storm ID. 
* Extract storm information: For each matched blackout event, extract relevant storm characteristics from the IBTrACS dataset, such as wind speed, pressure, and speed.


In [ ]:
BLACKOUT = "blackout.csv"
IBTRACS = "ibtracs_formatted"
JOINCOLS = ["NAME", "SEASON", "WMO_WIND", "STORM_SPEED", "LON", "LAT"]

blackout = pd.read_csv(os.path.join(WD, BLACKOUT))
ibtracs = pd.read_csv(os.path.join(WD, IBTRACS))
ibtracs = ibtracs.set_index("SID")
blackout.head()

In [ ]:
# Process times from sid column
blackout['landing_time'] = blackout['sid'].str.split("_").str[1]
blackout['landing_year'] = blackout['landing_time'].str[:4]
blackout['landing_month'] = blackout['landing_time'].str[4:6]
blackout['landing_day'] = blackout['landing_time'].str[6:8]
blackout['landing_hour'] = blackout['landing_time'].str[8:10]
blackout.head()


In [ ]:
# now match based-on SID and extract name, season
blackout['SID'] = blackout['sid'].str.split("_").str[0]
blackout = blackout.set_index("SID")

In [ ]:
blackout = blackout.join(ibtracs[JOINCOLS])
blackout = blackout.rename(columns={"NAME": "storm", "SEASON": "season"})

## Notes
There are a number of ways to calculate the distances between two geospatial locations. The simplest is to use the `haversine` library. If this is not available, the haversine distance can be computed manually. This can also be achieved with `geopandas`.

In [ ]:
def calculate_distances_haversine(blackout):
    from haversine import haversine

    def haversine_dist(coords):
        lat1, lon1, lat2, lon2 = coords
        return haversine((lat1, lon1), (lat2, lon2), unit='km')
    
    blackout = blackout.copy()
    blackout['distToTrack'] = blackout[['lat', 'lon', 'LAT', 'LON']].apply(haversine_dist, axis=1)
    blackout['distToTrack'] = blackout['distToTrack'] * 1000 # convert to meters
    return blackout


def calculate_distances_manual(blackout):
    from math import radians, cos, sin, asin, sqrt

    def haversine_dist(coords):
        """https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points"""
        lat1, lon1, lat2, lon2 = coords
        R = 6372.8 # this is in km.  For Earth radius in kilometers use 3959.87433 miles

        dLat = radians(lat2 - lat1)
        dLon = radians(lon2 - lon1)
        lat1 = radians(lat1)
        lat2 = radians(lat2)

        a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
        c = 2*asin(sqrt(a))

        return R * c
    
    blackout = blackout.copy()
    blackout['distToTrack'] = blackout[['lat', 'lon', 'LAT', 'LON']].apply(haversine_dist, axis=1)
    blackout['distToTrack'] = blackout['distToTrack'] * 1000 # convert to meters
    return blackout


def calculate_distances_with_geopandas(blackout):
    import geopandas as gpd

    columns = list(blackout.columns)
    blackout = gpd.GeoDataFrame(blackout)
    blackout['points_blackout'] = gpd.GeoSeries.from_xy(blackout['lon'], blackout['lat']).set_crs(4326)
    blackout['points_ibtracs'] = gpd.GeoSeries.from_xy(blackout['LON'], blackout['LAT']).set_crs(4326)

    # switch to Pseudo Mercator (in metres)
    blackout['points_blackout'] = blackout['points_blackout'].to_crs(3857)
    blackout['points_ibtracs'] = blackout['points_ibtracs'].to_crs(3857)

    blackout['distToTrack'] = blackout['points_blackout'].distance(blackout['points_ibtracs'])
    blackout['distToTrack'] = blackout['distToTrack']

    # geopandas doesn't handle NaNs well
    nan_mask = blackout[['lat', 'lon', 'LAT', 'LON']].isnull().any(axis=1)
    blackout.loc[nan_mask, 'distToTrack'] = pd.NA

    return blackout[columns + ['distToTrack']]

# play around with these
distance_func = calculate_distances_with_geopandas

In [ ]:
# get distances
blackout = distance_func(blackout)
blackout['distToTrack'].describe()


In [ ]:
# aggregate over storm: maxwind, mean speed, min dist to track
storm_characteristics = blackout.groupby("SID").agg(
    windmax=("WMO_WIND", "max"),
    speed=("STORM_SPEED", "mean"),
    distToTrack=("distToTrack", "min")
)

storm_characteristics.head()

In [ ]:
blackout[['windmax', 'speed', 'distToTrack']] = storm_characteristics
blackout

In [ ]:
# save processed blackout data
blackout = blackout.dropna(subset=['storm'])
blackout.to_csv(os.path.join(WD, "blackout_with_storms.csv"), index=False)

### Task III: Match socioeconomic data with blackout data
Socio-economic factors such as population density, income, and urbanization levels are critical for understanding the context of power outages. To incorporate this information into the dataset, follow these steps:
* Geographic matching: Use location identifiers, i.e., latitude and longitude, to link socio-economic data with the blackout dataset.
* Handle missing socio-economic data: Identify missing socio-economic values in the matched dataset. Apply approximation techniques to fill in the empty data.


In [ ]:
# reload the blackout data as a geospatial dataframe this time
blackout = pd.read_csv(os.path.join(WD, "blackout_with_storms.csv"))
blackout = gpd.GeoDataFrame(blackout, geometry=gpd.points_from_xy(blackout.lon, blackout.lat))
blackout = blackout.set_crs(4326)

#### Socioeconomic data
Versions of geopandas ≤ 1.0 have a `geopandas.datasets` module which allows us to import Natural Earth socioeconomic data directly to Python. Newer versions [have deprecated this module](https://github.com/geopandas/geopandas/issues/2751) but the country-level data (Admin 0 - Countries) can be downloaded from the [Natural Earth downloads page](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/).

In [ ]:
# method 1
SOCIOFILE = "ne_110m_admin_0_countries"
SOCIOCOLS = ['pop_est', 'continent', 'name', 'iso3', 'gdp_md_est', 'income', 'geometry']

socio = gpd.read_file(os.path.join(WD, SOCIOFILE))
print(list(socio.columns))
print(list(socio['INCOME_GRP'].unique()))

In [ ]:
# give the income groups simpler names
income_groups = {1: 'H', 2: 'H', 3: 'UM', 4: 'LM', 5: 'L'}
socio['INCOME_GRP'] = socio['INCOME_GRP'].str.split('.').str[0].astype(int)
socio['INCOME_GRP'] = socio['INCOME_GRP'].map(income_groups)


In [ ]:
# method 1
rename_columns = {'NAME': 'name', 'CONTINENT': 'continent', 'POP_EST': 'pop_est',
                  'GDP_MD': 'gdp_md_est', 'INCOME_GRP': 'income', 'ISO_A3': 'iso3'}

socio = socio.rename(columns=rename_columns)
socio = socio[SOCIOCOLS].copy()
socio.head()

In [ ]:
# check for rows with missing values
socio[socio.isnull().any(axis=1)]

Finally, we will do a spatial join, to match each blackout location to a country.

In [ ]:
# good practice to check crs alignment before geospatial ops
assert socio.crs == blackout.crs

if False: # make True to visualise, a little slow
    fig, ax = plt.subplots(figsize=(10, 4))
    socio.boundary.plot(ax=ax, color='black')
    blackout.plot(ax=ax, color='red', markersize=5)
    fig.suptitle('Blackout locations and country boundaries')

In [ ]:
# do the spatial join and save
blackout = gpd.sjoin(blackout, socio, how="left", predicate='within')
blackout = blackout.drop(columns=['index_right', 'geometry'])

blackout.to_csv(os.path.join(WD, "blackout_with_socio.csv"), index=False)


### Appendix: Wind factors
> Scale and shift used by agencies to convert their internal Dvorak 1-minute sustained winds to the officially reported values that are in IBTrACS. From Table 1 in  Knapp, K.R. & Kruk, M.C. (2010): Quantifying Interagency Differences in Tropical Cyclone Best-Track Wind Speed Estimates. Monthly Weather Review 138(4): 1459–1473. https://journals.ametsoc.org/view/journals/mwre/138/4/2009mwr3123.1.xml 

https://climada-python.readthedocs.io/en/stable/_modules/climada/hazard/tc_tracks.html

In [ ]:
IBTRACS_AGENCY_1MIN_WIND_FACTOR = {
    "usa": [1.0, 0.0],
    "tokyo": [0.60, 23.3],
    "newdelhi": [1.0, 0.0],
    "reunion": [0.88, 0.0],
    "bom": [0.88, 0.0],
    "nadi": [0.88, 0.0],
    "wellington": [0.88, 0.0],
    'cma': [0.871, 0.0],
    'hko': [0.9, 0.0],
    'ds824': [1.0, 0.0],
    'td9636': [1.0, 0.0],
    'td9635': [1.0, 0.0],
    'neumann': [0.88, 0.0],
    'mlc': [1.0, 0.0],
}
""""""


IBTRACS_USA_AGENCIES = [
    'atcf', 'cphc', 'hurdat_atl', 'hurdat_epa', 'jtwc_cp', 'jtwc_ep', 'jtwc_io',
    'jtwc_sh', 'jtwc_wp', 'nhc_working_bt', 'tcvightals', 'tcvitals'
]

IBTRACS_USA_1MIN_WIND_FACTOR = {
    agency: IBTRACS_AGENCY_1MIN_WIND_FACTOR['usa'] for agency in IBTRACS_USA_AGENCIES
    }
WIND_FACTORS = IBTRACS_AGENCY_1MIN_WIND_FACTOR | IBTRACS_USA_1MIN_WIND_FACTOR

wind_factors = pd.DataFrame(WIND_FACTORS).T
wind_factors.columns = ["scale", "shift"]
wind_factors.index.name = "agency"
wind_factors.to_csv(os.path.join(WD, "wind_factors.csv"))